In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install transformers bitsandbytes peft datasets evaluate

In [ ]:
import pandas as pd
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, DataCollatorForSeq2Seq
from datasets import Dataset

# Load data
df = pd.read_csv("/kaggle/input/sentiment/sentiment_data.csv")
dataset = Dataset.from_pandas(df)

# Split dataset
dataset = dataset.train_test_split(test_size=0.2)

# Model and tokenizer loading
base_model_name = "unsloth/llama-3-8b-bnb-4bit"

device_map = {
    "model": "auto",  # Automatically assign layers to GPU if possible
    "quantizer": "cpu"  # Offload the quantizer to the CPU if GPU memory is insufficient
}

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map=device_map,  # Use the custom device map
    torch_dtype="float16",
    quantization_config=bnb_config
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    load_in_8bit_fp32_cpu_offload=True  # Enable CPU offloading in FP32
)

model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id

# Tokenization and Data Collation
def tokenize_function(examples):
    return tokenizer(examples['Text'], padding='max_length', truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("Label", "labels")

# Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding="longest")

# LoRA Config
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

# Prepare model for training and apply PEFT
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# Print trainable parameters
model.print_trainable_parameters()

# Training Arguments
training_args = TrainingArguments(
    output_dir="./llama-fine-tuned",
    num_train_epochs=1,
    per_device_train_batch_size=8,  
    gradient_accumulation_steps=8,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    evaluation_strategy="epoch",
    save_strategy="epoch"
)

# Trainer Initialization
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Train the model
trainer.train()

# Evaluation
eval_results = trainer.evaluate()
print(f"Accuracy: {eval_results['eval_accuracy']:.4f}")